In [1]:
import numpy as np
import pandas as pd
import os
import glob
import EPMS
from pathlib import Path
import import_ipynb
# import train

importing Jupyter notebook from train.ipynb
importing Jupyter notebook from env_setup.ipynb
importing Jupyter notebook from datasets.ipynb
importing Jupyter notebook from lstm_predictor.ipynb
= MODELO = 

LSTM(88, 128, num_layers=4, batch_first=True, bidirectional=True)


Linear(in_features=256, out_features=88, bias=False)

Dataset length: 1 songs.
[Train Starting]


KeyboardInterrupt: 

In [2]:
RESOLUTION = 16
KEYBOARD_SIZE = 88
KEYBOARD_OFFSET = 21

SETTINGS = pd.Series(
  {
    'RESOLUTION': RESOLUTION, # Frame per measure amount
    'KEYBOARD_SIZE': KEYBOARD_SIZE, # Amount of keys in the Model's Keyboard
    'KEYBOARD_OFFSET': KEYBOARD_OFFSET # MIDI index for the first Model's Keyboard key
  }
)

In [3]:
def get_performance(data, first_frame, last_frame, to_float=True):
    stackframe = data.iloc[first_frame:last_frame, data.shape[1] - KEYBOARD_SIZE:]
    stackframe = stackframe.to_numpy()
    # if to_float:
    #     stackframe.astype(float)
    #     stackframe = stackframe + 0.0
    return stackframe

def get_info(data, first_frame, last_frame):
    infos = data.iloc[first_frame:last_frame, :data.shape[1] - KEYBOARD_SIZE]
    return infos

##### Merge Stackframe and Info to match the codification

In [20]:
def merge_data(infos, stackframe, instrument="Piano"):
    performance_df = pd.DataFrame(stackframe, columns=NOTES_COLUMNS)

    df = pd.concat([infos.reset_index(),performance_df],axis=1)

    # isso aqui resolve o problema de gerar um numero de measures maior que
    # o tamanho do contexto, ja que ele vai fazer um fill da info, uma vez que
    # nao estamos gerando ela
    # daria para colocar um if verificando se a info tem menos linhas,
    # mas nao faz diferença
    df = df.fillna(method="ffill").set_index("NAME")

    return df

##### Read all dataset and isolates the data from a single instrument

In [5]:
def create_instrument_performance_dataset(datasetPath, instrument, resolution):
    instrumentPerformanceDataset = []
    interpreted_files = glob.glob(f'{datasetPath}**/**.pkl', recursive=True)
    for int_file in interpreted_files:

        int_file_path = Path(int_file)
        interpreted_file = pd.read_pickle(int_file_path)
        # print(interpreted_file.head())

        interpreted_instrument = interpreted_file[interpreted_file.index==instrument]
        if(interpreted_instrument.empty):
            continue
        # print(interpreted_instrument.head())

        first_frame = 0
        last_frame = len(interpreted_instrument)

        interpreted_performance = get_performance(interpreted_instrument, first_frame, last_frame)
        # print(interpreted_performance.head())

        instrumentPerformanceDataset.append(np.array(interpreted_performance))

    return instrumentPerformanceDataset


In [ ]:
# datasetName = 'Pop36'
#
# modelType = 'prediction'
# modelPath  = f'Models/{modelType}/{datasetName}/'
# os.makedirs(modelPath, exist_ok=True)
#
#
#
# instrumentPerformanceDataset = createInstrumentPerformanceDataset(datasetName, INSTRUMENT, SETTINGS.RESOLUTION)
# print(f'Dataset {datasetName}, Instrument {INSTRUMENT}, Instrument Performance Dataset size {len(instrumentPerformanceDataset)}')

In [6]:
# INSTRUMENT = 'Piano'
# Read context file
CONTEXT = pd.read_pickle("C_major_scale/C_major_scale.pkl")

# Filter by instrument
# CONTEXT = CONTEXT[CONTEXT.index==INSTRUMENT]

# If the file does not contain the instrument
if len(CONTEXT) == 0:
    raise ValueError(f'Oops, it seems that the selected context file does not contain this instrument ({INSTRUMENT}).')

# print(f'{INSTRUMENT} context data shape {CONTEXT.shape}')
# print(f'{INSTRUMENT} context measures {int(CONTEXT.shape[0] / SETTINGS.RESOLUTION)}')

##### Set generation parameters

In [22]:
CONTEXT_FIRST_MEASURE = 1
CONTEXT_AMOUNT = 8 # Amount of measures of context
GENERATE_AMOUNT = 4 # Amount of measures to generate
TEMPERATURE = 0.5 # Predicted probability must be >= 80% to note be played
BEATS_MEASURE = int(CONTEXT.TS[0][0]) # Amount of beats in a measure
MEASURE_LEN = SETTINGS.RESOLUTION * BEATS_MEASURE

first_context_frame = (CONTEXT_FIRST_MEASURE-1)*MEASURE_LEN
last_context_frame = first_context_frame + MEASURE_LEN*CONTEXT_AMOUNT

first_generated_frame = last_context_frame
last_generated_frame = first_generated_frame + MEASURE_LEN*GENERATE_AMOUNT

context_info = get_info(CONTEXT, first_context_frame, last_context_frame)
context_performance = get_performance(CONTEXT, first_context_frame, last_context_frame, to_float=True)

generated_info = get_info(CONTEXT, first_generated_frame, last_generated_frame)

# print(f'CONTEXT_INFO: \n {context_info} \n\n')
# print(f'CONTEXT_PERFOMANCE: \n {context_performance} \t Shape: {context_performance.shape}')
#
# print(f'GENERATED_INFO: \n {generated_info}')

In [23]:
NOTES_COLUMNS = CONTEXT.columns[CONTEXT.shape[1] - KEYBOARD_SIZE:]

df = merge_data(context_info, context_performance)

print(df.to_markdown())

| NAME                              | INSTRUMENT   |   MIDI_PROGRAM | SOUND          |   MEASURE |   BEAT |   FRAME | ORIGINAL_KS   | TS   |   TEMPO | A0    | B-0   | B0    | C1    | C#1   | D1    | E-1   | E1    | F1    | F#1   | G1    | G#1   | A1    | B-1   | B1    | C2    | C#2   | D2    | E-2   | E2    | F2    | F#2   | G2    | G#2   | A2    | B-2   | B2    | C3    | C#3   | D3    | E-3   | E3    | F3    | F#3   | G3    | G#3   | A3    | B-3   | B3    | C4    | C#4   | D4    | E-4   | E4    | F4    | F#4   | G4    | G#4   | A4    | B-4   | B4    |       C5 | C#5   |       D5 | E-5   |       E5 |       F5 | F#5   |       G5 | G#5   |       A5 | B-5   |       B5 |       C6 | C#6   | D6    | E-6   | E6    | F6    | F#6   | G6    | G#6   | A6    | B-6   | B6    | C7    | C#7   | D7    | E-7   | E7    | F7    | F#7   | G7    | G#7   | A7    | B-7   | B7    | C8    |
|:----------------------------------|:-------------|---------------:|:---------------|----------:|-------:|--------:|:---

In [9]:
print("Deserializing")
deserialized = EPMS.deserialization.file(df, SETTINGS)
print("Done")


Deserializing
Done
